In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
class args:
    dataset_name='Cora' 
    val_pct=0.1
    test_pct=0.2
    num_negs = 1
a = args()

In [9]:
from src.data import get_data
data = get_data(a)

reading data from: /data_sda/dataset/Cora


/home/lyl/miniconda3/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [10]:
data[1]['train']

Data(x=[2485, 1433], edge_index=[2, 7100], y=[2485], train_mask=[2485], test_mask=[2485], val_mask=[2485], edge_label=[7100], edge_label_index=[2, 7100])

In [11]:
data[1]['train'].edge_index

tensor([[ 515, 1245,  666,  ...,  204, 2242,  880],
        [1390, 1601, 2145,  ...,  110,   37,  729]])

In [ ]:
import torch
from torch_geometric.data import Data
from torch_scatter import scatter

def wl_node_coloring_with_degree(data, max_iters=10):
    # Calculate node degrees
    edge_index = data.edge_label_index
    num_nodes = data.num_nodes
    node_degrees = scatter(torch.ones_like(edge_index[0]), edge_index[1], dim=0, dim_size=num_nodes, reduce='sum')
    
    # Initialize colors with node degrees
    node_colors = node_degrees.clone()

    for _ in range(max_iters):
        # Aggregate colors of neighbors
        aggregated_colors = scatter(node_colors[edge_index[0]], edge_index[1], dim_size=num_nodes, reduce='mean')
        
        # Concatenate current colors with aggregated neighbor colors
        new_colors = torch.stack([node_colors, aggregated_colors], dim=-1)
        
        # Apply a simple hash function: generate unique integers from concatenated values
        new_colors = torch.sum(new_colors * torch.tensor([1000, 1], device=data.x.device), dim=-1)
        
        # Check for convergence
        if torch.equal(node_colors, new_colors):
            break
        node_colors = new_colors

    data.node_colors = node_colors
    return data

def edge_marking(data):
    # Get final node colors
    node_colors = data.node_colors
    
    # For each edge, compare the colors of the connected nodes
    edge_u = data.edge_label_index[0]
    edge_v = data.edge_label_index[1]
    edge_marks = (node_colors[edge_u] == node_colors[edge_v]).long()  # 1 if same, 0 otherwise
    
    data.edge_marks = edge_marks
    return data

def compute(data):
    colors = wl_node_coloring_with_degree(data[1]['train'])
    data = edge_marking(colors)
    return None
compute(data)

# Now `data.edge_marks` contains the edge marks (1 for same color, 0 for different).


In [18]:
data

(Cora(),
 {'train': Data(x=[2485, 1433], edge_index=[2, 7100], y=[2485], train_mask=[2485], test_mask=[2485], val_mask=[2485], edge_label=[7100], edge_label_index=[2, 7100], node_colors=[2485], edge_marks=[7100]),
  'valid': Data(x=[2485, 1433], edge_index=[2, 7100], y=[2485], train_mask=[2485], test_mask=[2485], val_mask=[2485], edge_label=[1012], edge_label_index=[2, 1012]),
  'test': Data(x=[2485, 1433], edge_index=[2, 8112], y=[2485], train_mask=[2485], test_mask=[2485], val_mask=[2485], edge_label=[2026], edge_label_index=[2, 2026])},
 False,
 'hits')

In [ ]:
data.node_colors

In [19]:
data[1]['train'].edge_marks.sum()

tensor(120)

In [ ]:
data